<h1><center><font color = 'blue'>Prepare ML data 4 nearest pts</font></center></h1>

In [1]:
import multiprocessing as mp
import pandas as pd
import numpy as np
import sagemaker
import boto3
import glob
import time
import os
import re

from itertools import product
from math import trunc

In [2]:
!conda install -y xarray netcdf4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.5.11

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - netcdf4
    - xarray


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |           py36_1         140 KB
    libgcc-ng-8.2.0            |       hdf63c60_1         7.6 MB
    numpy-1.14.3               |   py36h28100ab_1          41 KB
    xarray-0.10.8              |           py36_0         730 KB
    netcdf4-1.4.1              |   py36h4b4f87f_0         532 KB
    hdf4-4.2.13                |       h3ca952b_2         916 KB
    cftime-1.0.0b1             |   py36h035aef0_0         260 KB
    libnetcdf-4.6.1            |       h13459d8_0         1.2 MB


In [3]:
import multiprocessing as mp
import xarray as xr
import pandas as pd
import numpy as np
import sagemaker
import boto3
import glob
import time
import os
import re

from itertools import product
from math import trunc

In [4]:
# define a function that import level names from a txt file
def get_levels(resource):
    filename = "levels.txt"
    resource.Bucket("fsoi").download_file(filename, filename)

    f = open(filename, "r")
    levels = f.read().split("\n")
    f.close()

    os.remove(filename)

    return levels


# define a function that creates lat, lon dicts which for an obs lat or lon
# return the two nearest in bgk
def get_coord_dicts():
    # get lat and lon bkg possible values
    lat_bkg = np.arange(-90, 90.5, 0.5)
    lon_bkg = np.arange(-180, 180.0, 0.625) # no 180 lon value in bkg
    
    # get lat and lon obs possible values
    lat_keys = np.arange(-90., 90.01, 0.01)
    lon_keys = np.arange(-180., 180.01, 0.01)
    
    # for each possible obs value get the 2 nearest bkg values
    lat_values = [sorted(lat_bkg, key=lambda x: abs(x - lat))[:2] for lat in lat_keys]
    lon_values = [sorted(lon_bkg, key=lambda x: abs(x - lon))[:2] for lon in lon_keys]
    
    # join keys and values into dict
    lat_dict = dict(zip(np.round(lat_keys, 2), lat_values))
    lon_dict = dict(zip(np.round(lon_keys, 2), lon_values))
    
    return lat_dict, lon_dict


# define a function that loads obs and bkg data into pandas dataframes
def load_data(filepath, date):
    resource = boto3.resource("s3")
    filename = "/tmp/" + filepath.split("/")[-1]
    resource.Bucket("fsoi").download_file(filepath, filename)
    
    obs = pd.read_hdf(filename).xs(["AMSUA_N18", 7], level=["PLATFORM", "CHANNEL"])
    os.remove(filename)
    
    obs = obs.reset_index(level=[0, 1])
    obs = obs.drop(["OBTYPE", "OBERR", "PRESSURE"], axis=1)
    
    # fix lon between -180/180 instead of 0/360
    mask_lon = obs[obs["LONGITUDE"] > 180].index.tolist()
    obs.loc[mask_lon, "LONGITUDE"] = obs.loc[mask_lon, "LONGITUDE"] - 360
    
    date = date[:-2] + "_" + date[-2:]
    month = date[:4] + "_" + date[4:6] + "/"
    filename = "e5130_hyb_01.bkg.eta." + date + "z.nc4"
    filepath = "bkg/" + month + filename
    filename = "/tmp/" + filename
    resource.Bucket("fsoi").download_file(filepath, filename)
    
    bkg = xr.open_dataset(filename).to_dataframe()\
            .reset_index(level=[0, 1, 2, 3]).drop('time', axis=1)
    
    os.remove(filename)
            
    obs = obs.sort_values(["LONGITUDE", "LATITUDE"]).reset_index(drop=True)
    bkg = bkg.sort_values(["lon", "lat"]).reset_index(drop=True)
    
    # fix bkg zeros lat and lon to 0.0 instead of something like -1.79751e-13
    zero_lat_mask = bkg[(bkg['lat'] < 0.1) & (bkg['lat'] > -0.1)].index.tolist()
    zero_lon_mask = bkg[(bkg['lon'] < 0.1) & (bkg['lon'] > -0.1)].index.tolist()
    
    bkg.loc[zero_lat_mask, 'lat'] = 0.0
    bkg.loc[zero_lon_mask, 'lon'] = 0.0
    
    bkg.set_index(['lat', 'lon'], inplace=True)
        
    return obs, bkg


# define a function that for an obs append a list with the 4 nearest bkg pts
def get_nearest_pts(obs_row, nearest_pts, lat_dict, lon_dict):
    lat = np.round(obs_row.LATITUDE, 2)
    lon = np.round(obs_row.LONGITUDE, 2)
    
    row_nearest_pts = [
            (lat_dict[lat][i], lon_dict[lon][j])
            for i, j in product([0, 1], [0, 1])
            ]
        
    nearest_pts.append(row_nearest_pts)
    
    return None


# define a function that gets bkg levels data for a particular point
def get_lev_data(bkg, ix, level_cols):
    return bkg.loc[ix:ix + 71, level_cols[1:]].copy().stack().reset_index(drop=True)


# define a function that adds level data from bkg 3D to 2D by transposing it
def add_lev_data(bkg_3D, bkg_2D, obs, level_cols, levels, i):
    lev_data = []
    
    bkg_2D.index.map(
            lambda ix: lev_data.append(get_lev_data(bkg_3D, ix, level_cols))
            )
    
    lev_data = pd.concat(lev_data, axis=1).transpose().reset_index(drop=True)
        
    lev_data.columns = [
            col + '_' + lev
            for lev in levels
            for col in level_cols[1:]
            ]
    
    bkg_2D = pd.concat([bkg_2D.reset_index(drop=True), lev_data], axis=1)
    bkg_2D = bkg_2D.add_prefix('point' + str(i + 1) + '_')
    
    return bkg_2D


# define a function that get bkg nearest data from observations
def get_nearest_bkg(obs, bkg, lat_dict, lon_dict, level_cols, levels):
    nearest_pts, nearest_bkg = [], []
    
    obs.apply(lambda row: get_nearest_pts(row, nearest_pts, lat_dict,
                                          lon_dict), axis=1)
    
    pts = [
            [points[i] for points in nearest_pts]
            for i in range(0, len(nearest_pts[0]))
            ]
    
    for i, pts_i in enumerate(pts):
        bkg_3D_i = bkg.loc[pts_i].reset_index(level=[0, 1])
        
        mask_2D = np.arange(0, len(bkg_3D_i), 72)
        bkg_2D_i = bkg_3D_i.drop(level_cols, axis=1).loc[mask_2D, :]
        bkg_2D_i = add_lev_data(bkg_3D_i, bkg_2D_i, obs, level_cols, levels, i)
        
        nearest_bkg.append(bkg_2D_i)
        
    nearest_bkg = pd.concat(nearest_bkg, axis=1)
    
    return nearest_bkg


# define a function that merges our training data from all files
def merge_train_data(filepaths, i, n, lat_dict, lon_dict, levels):
    level_cols = ["lev", "delp", "u", "v", "tv", "sphu", "ozone", "qitot", "qltot"]
    ml_data = pd.DataFrame()
    
    for j in range(i, len(filepaths), n):
        
        start = time.time()
        
        filepath = filepaths[j]
        date = re.findall("[0-9]+.h5", filepath)[0][:-3]
        obs, bkg = load_data(filepath, date)
        
        nearest_bkg = get_nearest_bkg(obs, bkg, lat_dict, lon_dict, level_cols,
                                      levels)
        
        merge = pd.concat([obs, nearest_bkg], axis=1)
        ml_data = pd.concat([ml_data, merge], axis=0)
        
        end = time.time()
        print("Merge {} obs and bkg done in: {} min and {} sec".format(
            date, trunc((end - start)/60), round((end - start)%60)
        ))
        
    return ml_data

In [5]:
pstart = time.time()

client = boto3.client("s3")
resource = boto3.resource("s3")

lat_dict, lon_dict = get_coord_dicts()
levels = get_levels(resource)

months = ["2014_12", "2015_01", "2015_02"]
filepaths = []

for month in months:
    monthpaths = client.list_objects(Bucket="fsoi", Prefix="obs/GMAO_" + month + "/GMAO.dry.")
    filepaths += [dic["Key"] for dic in monthpaths["Contents"]]

# define how many process we want to run
n = 18
pool = mp.Pool(processes=n)

# run our processes
results = [
        pool.apply_async(merge_train_data, args=(filepaths[248:], i, n, lat_dict, # PERFORM MONTH BY MONTH TO AVOID 
                                                 lon_dict, levels))               # MEMORY ERROR(too large to unpickle)
        for i in range(n)
        ]

results = [p.get() for p in results]

# merge our training data
ml_data = pd.concat(results, axis=0)

ml_data = ml_data.sort_values(["DATETIME", "LATITUDE", "LONGITUDE"])\
                 .reset_index(drop=True)

# save and compress training data in hdf5 format
start = time.time()
                             
ml_data.to_hdf("../efs/data/amsua_n18_ch7_4pts_part3.h5", key="df", complevel=9)

end = time.time()
print("Saved and compressed in: {} min and {} sec".format(trunc((end - start)/60),
                                                          round((end - start)%60)))
    
# display total program time
pend = time.time()
print("Total program took: {} hours and {} min".format(trunc((pend - pstart)/3600),
                                                       round((pend - pstart)%3600/60)))

Merge 2015020206 obs and bkg done in: 1 min and 58 sec
Merge 2015020212 obs and bkg done in: 1 min and 58 sec
Merge 2015020118 obs and bkg done in: 2 min and 1 sec
Merge 2015020112 obs and bkg done in: 2 min and 6 sec
Merge 2015020406 obs and bkg done in: 2 min and 3 sec
Merge 2015020100 obs and bkg done in: 2 min and 10 sec
Merge 2015020106 obs and bkg done in: 2 min and 22 sec
Merge 2015020300 obs and bkg done in: 3 min and 13 sec
Merge 2015020200 obs and bkg done in: 3 min and 19 sec
Merge 2015020312 obs and bkg done in: 3 min and 26 sec
Merge 2015020400 obs and bkg done in: 3 min and 55 sec
Merge 2015020700 obs and bkg done in: 2 min and 7 sec
Merge 2015020512 obs and bkg done in: 2 min and 1 sec
Merge 2015020818 obs and bkg done in: 2 min and 3 sec
Merge 2015020506 obs and bkg done in: 4 min and 9 sec
Merge 2015020618 obs and bkg done in: 2 min and 19 sec
Merge 2015020518 obs and bkg done in: 2 min and 2 sec
Merge 2015020218 obs and bkg done in: 4 min and 22 sec
Merge 2015020418 o